# vector swap



In [1]:
from generate import *
from closest_vs_minimal import *
from sage.modules.free_module_integer import IntegerLattice
import time 

[0, 1, 0] tuple
(-4, 5, 2) this vector
(-1, 1, 2)
(-5, 6, 4) this vector

[1, 0, 0] tuple
(-4, 5, 2) this vector
(8, -7, -5)
(4, -2, -3) this vector

[1, 1, 0] tuple
(-4, 5, 2) this vector
(8, -7, -5)
(-1, 1, 2)
(3, -1, -1) this vector

efficient returned:  [(-4, 5, 2), (-5, 6, 4), (4, -2, -3), (3, -1, -1)]
   (3, -1, -1)
   (4, -2, -3)
   (-4, 5, 2)
   (-5, 6, 4)


In [2]:
def gaussian_reduction(b_1, b_2):
    """Input: vectos as a list
    Output: sage vectors [longer, shorter]
    """
    b_1 = vector(b_1)
    b_2 = vector(b_2)
    if b_1.norm() < b_2.norm():
        b_1 , b_2 = b_2, b_1
    r = round(b_1.dot_product(b_2) / b_2.norm()**2)
    a = b_1 - r * b_2
    if a.norm() < b_2.norm():
        a, b_2 = gaussian_reduction(a, b_2)
    return a, b_2


In [3]:
from sage.modules.free_module_integer import IntegerLattice
from generate import *

def vector_to_int_list(vector) -> list:
    return [int(num) for num in vector.list()]

def semaev(basis):
    dim = basis.nrows()

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterace = 0
    shortest_vectors_sequence = []
    avg_norms = []
    while True:
        # avg_norms.append(mean([b.norm().n() for b in basis]))
        shortest_vectors_sequence.append(basis[0].norm().n())
        iterace += 1
        # print(basis)
        # reduce the pair b_1 and b_2
        start = time.time()

        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])
        # print("gauss", time.time()-start)

        # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        start = time.time()
        G = gram_matrix(basis)
        # print("gram", time.time()-start)

        start = time.time()
        lc_cube = find_real_minimum_mistni(G, dim-1, 1) # FIXME might break for nonsquare matrices
        # print("frm", time.time()-start)


        start = time.time()
        slc = shortest_lc_in_cube(lc_cube, basis)
        # print("slc", time.time()-start,"\n")
        candidate = vector(ZZ,slc) * basis
        # candidate_norm = sqrt((vector(slc)*G*matrix(slc)-transpose())[0]).n()
        # print(candidate_norm, candidate.norm().n(), "slcG slc.T vs kandidat norma, melo by byt stejne")

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()
        # print(candidate.norm().n() < longest_norm , (candidate) not in basis, candidate.norm(), longest_norm)
        if candidate.norm().n() >= longest_norm.n():
            break
        # if candidate.norm().n() < longest_norm.n() and (candidate) not in basis:
        basis[dim-1] = candidate
        sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
        basis = basis[sorted_indices, :]
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    # ratios = [shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1] for i in range(len(shortest_vectors_sequence)-1)]
    # print(avg_norms)
    return basis, iterace

def find_real_minimum_mistni(G, current_row, inserted_component) -> vector:
    dimension = len(matrix_to_list(G))
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right(-(inserted_component) * matrixB)).list()
    result.insert(current_row, inserted_component)
    return vector(result).n()




In [4]:
# # # # TADY POČÍTÁM POČET ITERACÍ VERSUS VELIKOST ČÍSEL MATII 

# for exp in range(5, 45, 5):
#     max_it = 0
#     min_it = math.inf
#     iterations = 100
#     prime = random_prime(10^6, True, 10)
#     for j in range(iterations):
#         this_it = semaev(random_special_matrix(3, 10^exp))[1]
#         if this_it > max_it:
#             max_it = this_it
#         if this_it < min_it:
#             min_it = this_it
#     print("Range čísel v prvním sloupci: deset na ", exp, "-> min iterací", min_it, "max iterací", max_it)


In [5]:
# ZDE POČÍTÁM ÚSPĚŠNOST  +  RUNTIME SEMAIEVA


from math import isclose
import time

def counter(dimension : int, perimeter, tries, function):
    print("dimension:", dimension)
    res = {"True":0, "False":0}
    total_time =0
    total_iterations = 0
    better_than_LLL_count = 0
    diff_from_exact=0
    for _ in range(tries):
        mojematice = random_special_matrix(dimension, perimeter)
        # mojematice = randomMatrix(dimension, perimeter)
        start = time.time()
        swapresult, this_it = function(mojematice)
        total_iterations += float(this_it)
        end = time.time()
        total_time += end - start
        this_function_closest_norm = euclidean_norm(matrix_to_list(swapresult)[0])
        L = IntegerLattice(mojematice, lll_reduce=False)
        sv_exact = L.shortest_vector()
        sv_exact_norm = sv_exact.norm().n()
        lllnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(mojematice.LLL())))

        # if this_function_closest_norm < lllnorm:
        #     # print(" better than LLL!", this_function_closest_norm, sv_exact_norm, lllnorm)
        #     better_than_LLL_count +=1

        if  isclose(this_function_closest_norm, sv_exact_norm, abs_tol=1e-4):
            res["True"]+=1
            continue

        # print(mojematice)
        # print("falsuje", this_function_closest_norm,"exact: ", sv_exact_norm, "LLLko je ", lllnorm, "B = ", matrix_to_list(mojematice))
        diff_from_exact += (this_function_closest_norm - sv_exact_norm)
        res["False"]+=1
    print("Total time:", total_time/tries, "\t\t Average num of iterations", total_iterations/tries,"\t\t total diff from exact, if failed",diff_from_exact,"\t\tof method ", function)
    return res

In [6]:
def first_nonzero_norm_vector(vectors):
    for v in vectors:
        if euclidean_norm(v) != 0:
            return v

In [7]:
def semaev_modified_for_higher_bases(basis):
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterace = 0
    shortest_vectors_sequence = []

    while True:
        iterace += 1
        # shortest_vectors_sequence.append(basis[0].norm().n())

        # Reduce the pair b_1 and b_2
        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

        # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis)
        lc_cube = vector_to_list(find_real_minimum_mistni(G, dim - 1, 1))
        slc = shortest_lc_in_cube(lc_cube, basis)
        candidate = vector(ZZ,slc) * basis
        candidate_norm = candidate.norm().n() 

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()

        # if I found a shorter vector a, continue
        if candidate_norm < longest_norm:
            # print(basis,"\n")
            # print("checkpoint C")
            basis[dim-1] = candidate
            sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
            basis = basis[sorted_indices, :]
            continue
        else: # pokud to už nezkratim 
            # print("rekurzuju, aktualni sv ma normu ", basis[0].norm().n())
            tmp_base, tmp_it = semaev(basis[0:dim-1]) # FIXME dat zpět trojku
            iterace += tmp_it

            # semaev 3x3 didnt change anything
            if basis[0:dim-1] == tmp_base:
                break
            basis[0:dim-1] = tmp_base
            # shortest_vectors_sequence = shortest_vectors_sequence + tmp_svc
            sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
            basis = basis[sorted_indices, :]
    # print(iterace, "iterace")
    # ratios = [shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1] for i in range(len(shortest_vectors_sequence)-1)]
    # print([v  for v in shortest_vectors_sequence])
    # print("-")
    # # print(ratios)
    # print("")
    return basis, iterace


# print(counter(4, 10^10, 1000, semaev_modified_for_higher_bases))




In [8]:
# print(counter(5, 10^10, 1000, semaev_modified_for_higher_bases))


In [9]:
def semaev_with_rounding(basis):
    dim = basis.nrows()

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterace = 0
    shortest_vectors_sequence = []

    while True:
        shortest_vectors_sequence.append(basis[0].norm().n())
        iterace += 1
        # print(basis)
        # reduce the pair b_1 and b_2
        start = time.time()

        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])
        # print("gauss", time.time()-start)

        # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        start = time.time()
        G = gram_matrix(basis)
        # print("gram", time.time()-start)

        start = time.time()
        lc_cube = find_real_minimum_mistni(G, dim-1, 1) # FIXME might break for nonsquare matrices
        # print("frm", time.time()-start)


        start = time.time()
        slc = vector([round(x) for x in lc_cube])
        # print(slc, lc_cube)
        # print("slc", time.time()-start,"\n")
        candidate = vector(ZZ,slc) * basis
        # candidate_norm = sqrt((vector(slc)*G*matrix(slc)-transpose())[0]).n()
        # print(candidate_norm, candidate.norm().n(), "slcG slc.T vs kandidat norma, melo by byt stejne")

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()
        # print(candidate.norm().n() < longest_norm , (candidate) not in basis, candidate.norm(), longest_norm)
        if candidate.norm().n() >= longest_norm.n():
            break
        # if candidate.norm().n() < longest_norm.n() and (candidate) not in basis:
        basis[dim-1] = candidate
        sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
        basis = basis[sorted_indices, :]
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    ratios = [shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1] for i in range(len(shortest_vectors_sequence)-1)]
    # print(list(ratios))
    return basis, iterace



In [10]:
# print(counter(6, 10^10, 100, semaev_with_rounding))
# print(counter(10, 10^10, 100, semaev_with_rounding))
# print(counter(15, 10^10, 100, semaev_with_rounding))


In [11]:
def jeden_semaev_a_pak_LLL(basis):
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    # -- - - - - - SEMAEV

    # reduce the pair b_1 and b_2
    basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

    # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
    G = gram_matrix(basis)
    lc_cube = find_real_minimum_mistni(G, dim-1, 1) # FIXME might break for nonsquare matrices
    slc = shortest_lc_in_cube(lc_cube, basis)
    candidate = vector(ZZ,slc) * basis

    # rovnou nahrazuju

    basis[dim-1] = candidate

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]
    
    # print("předlll")
    # for vec in basis:
    #     print(vec.norm().n())
        
    basis, it = poradne_LLL(basis)

    # print("po LLL")
    # for vec in basis:
    #     print(vec.norm().n())
        
    # basis = basis.LLL()
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    return basis, it

# jeden_semaev_a_pak_LLL(random_special_matrix(3,10000))


In [12]:
def jeden_semaev_round_tho_a_pak_LLL(basis):
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    # -- - - - - - SEMAEV

    # reduce the pair b_1 and b_2
    basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

    # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
    G = gram_matrix(basis)
    lc_cube = find_real_minimum_mistni(G, dim-1, 1) # FIXME might break for nonsquare matrices
    slc = vector([round(x) for x in lc_cube])
    candidate = vector(ZZ,slc) * basis

    # rovnou nahrazuju

    basis[dim-1] = candidate

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]
    
    # print("předlll")
    # for vec in basis:
    #     print(vec.norm().n())
        
    basis, it = poradne_LLL(basis)

    # print("po LLL")
    # for vec in basis:
    #     print(vec.norm().n())
        
    # basis = basis.LLL()
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    return basis, it

# jeden_semaev_a_pak_LLL(random_special_matrix(3,10000))


In [13]:
def vestavene_LLL(basis):
    dim = basis.nrows()
    basis = basis.LLL()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :] 
    return basis, 0
    

In [14]:
from LLL_implementation import full_LLL, full_LLL_printing
from generate import random_special_matrix

def poradne_LLL(basis):
    basis, it = full_LLL(basis)
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :] 
    return matrix(basis), it


In [15]:
# # print(counter(10, 10^5, 10))
# for exp in range(5,41,5):
#     print("exp",exp)
#     print(counter(3, 10^exp, 100, semaev_with_rounding))

# # print(counter(4, 10^10, 100, poradne_LLL))
# # print(counter(6, 10^10, 100, poradne_LLL))
# # print(counter(10, 10^10, 100, poradne_LLL))
# # print(counter(15, 10^10, 100, poradne_LLL))
# # print(counter(20, 10^10, 100, poradne_LLL))



In [18]:
for i in range(100):
    print(counter(4, 10^10, 100, semaev_modified_for_higher_bases))


dimension: 4
Total time: 0.2820459294319153 		 Average num of iterations 15.11 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.2918910217285156 		 Average num of iterations 15.6 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.2820504021644592 		 Average num of iterations 14.93 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.2776780366897583 		 Average num of iterations 15.27 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.29223057985305784 		 Average num of iterations 15.58 		 total diff from exact, if failed 0 		of method

In [17]:
for i in range(100):
    print(counter(4, 10^10, 100, semaev_modified_for_higher_bases))


dimension: 4
Total time: 0.4342323184013367 		 Average num of iterations 15.56 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.37125003814697266 		 Average num of iterations 15.93 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.34415124654769896 		 Average num of iterations 15.63 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.32569204568862914 		 Average num of iterations 15.72 		 total diff from exact, if failed 0 		of method  <function semaev_modified_for_higher_bases at 0xffff10527760>
{'True': 100, 'False': 0}
dimension: 4
Total time: 0.3209505534172058 		 Average num of iterations 15.68 		 total diff from exact, if failed 0 		of met